# Understanding EntitySets

https://featuretools.alteryx.com/en/stable/getting_started/using_entitysets.html

An `EntitySet` is a collection of dataframes and the relationships between them. They are useful for preparing raw, structured datasets for feature engineering. While many functions in Featuretools take `dataframes` and `relationships` as separate arguments, it is recommended to create an `EntitySet`, so you can more easily manipulate your data as needed.

## Setup

In [1]:
import featuretools as ft
import pandas

# Display options
pandas.set_option('display.max_rows', 10)

# Load data
data = ft.demo.load_mock_customer()
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
products_df = data["products"]

In [2]:
transactions_df

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,birthday
0,298,1,2014-01-01 00:00:00,5,127.64,2,desktop,2014-01-01 00:00:00,13244,2012-04-15 23:31:04,1986-08-18
1,2,1,2014-01-01 00:01:05,2,109.48,2,desktop,2014-01-01 00:00:00,13244,2012-04-15 23:31:04,1986-08-18
2,308,1,2014-01-01 00:02:10,3,95.06,2,desktop,2014-01-01 00:00:00,13244,2012-04-15 23:31:04,1986-08-18
3,116,1,2014-01-01 00:03:15,4,78.92,2,desktop,2014-01-01 00:00:00,13244,2012-04-15 23:31:04,1986-08-18
4,371,1,2014-01-01 00:04:20,3,31.54,2,desktop,2014-01-01 00:00:00,13244,2012-04-15 23:31:04,1986-08-18
...,...,...,...,...,...,...,...,...,...,...,...
495,112,35,2014-01-01 08:56:15,5,55.42,3,mobile,2014-01-01 08:44:20,13244,2011-08-13 15:42:34,2003-11-21
496,111,35,2014-01-01 08:57:20,3,34.87,3,mobile,2014-01-01 08:44:20,13244,2011-08-13 15:42:34,2003-11-21
497,276,35,2014-01-01 08:58:25,1,10.94,3,mobile,2014-01-01 08:44:20,13244,2011-08-13 15:42:34,2003-11-21
498,266,35,2014-01-01 08:59:30,5,19.86,3,mobile,2014-01-01 08:44:20,13244,2011-08-13 15:42:34,2003-11-21


In [3]:
products_df

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


## Creating an EntitySet

Pass `id` to the constructor to name the dataset.

In [4]:
es = ft.EntitySet(id = "customer_data")

To get started, we add the transactions dataframe to the EntitySet. In the call to add_dataframe, we specify three important parameters:

* The index parameter specifies the column that uniquely identifies rows in the dataframe.
* The time_index parameter tells Featuretools when the data was created.
* The logical_types parameter indicates that “product_id” should be interpreted as a Categorical column, even though it is just an integer in the underlying data.

In [6]:
from woodwork.logical_types import Categorical, PostalCode

es = es.add_dataframe(
    dataframe_name = "transactions",
    dataframe = transactions_df,
    index = "transaction_id",
    time_index = "transaction_time",
    logical_types = {
        "product_id": Categorical,
        "zip_code": PostalCode,
    },
)

es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 11]
  Relationships:
    No relationships

You can also use a setter on the EntitySet object to add dataframes.

`es["transactions"] = transactions_df`

This method associates each column in the dataframe to a Woodwork logical type. Each logical type can have an associated standard semantic tag that helps define the column data type. If you don’t specify the logical type for a column, it gets inferred based on the underlying data. The logical types and semantic tags are listed in the schema of the dataframe. For more information on working with logical types and semantic tags, take a look at the Woodwork documention.

In [7]:
es["transactions"].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
transaction_id,Integer,['index']
session_id,Integer,['numeric']
transaction_time,Datetime,['time_index']
product_id,Categorical,['category']
amount,Double,['numeric']
customer_id,Integer,['numeric']
device,Categorical,['category']
session_start,Datetime,[]
zip_code,PostalCode,['category']


In [8]:
es = es.add_dataframe(
    dataframe_name="products", dataframe=products_df, index="product_id"
)

es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    No relationships

## Adding a Relationship

With two dataframes in our EntitySet, we can add a relationship between them.

We want to relate these two dataframes by the columns called “product_id” in each dataframe. Each product has multiple transactions associated with it, so it is called the parent dataframe, while the transactions dataframe is known as the child dataframe. When specifying relationships, we need four parameters: the parent dataframe name, the parent column name, the child dataframe name, and the child column name.

Note that each relationship *must denote a one-to-many relationship* rather than a relationship which is one-to-one or many-to-many.

In [9]:
es = es.add_relationship("products", "product_id", "transactions", "product_id")

es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    transactions.product_id -> products.product_id

When working with raw data, it is common to have sufficient information to justify the creation of new dataframes. In order to create a new dataframe and relationship for sessions, we “normalize” the transaction dataframe.

In [10]:
es = es.normalize_dataframe(
    base_dataframe_name="transactions",
    new_dataframe_name="sessions",
    index="session_id",
    make_time_index="session_start",
    additional_columns=[
        "device",
        "customer_id",
        "zip_code",
        "session_start",
        "join_date",
    ],
)

es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 6]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id

If we look at the schema from the transactions dataframe and the new sessions dataframe, we see two more operations that were performed automatically:

In [11]:
es["transactions"].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
transaction_id,Integer,['index']
session_id,Integer,"['foreign_key', 'numeric']"
transaction_time,Datetime,['time_index']
product_id,Categorical,"['foreign_key', 'category']"
amount,Double,['numeric']
birthday,Datetime,[]


In [12]:
es["sessions"].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
session_id,Integer,['index']
device,Categorical,['category']
customer_id,Integer,['numeric']
zip_code,PostalCode,['category']
session_start,Datetime,['time_index']
join_date,Datetime,[]


* It removed “device”, “customer_id”, “zip_code” and “join_date” from “transactions” and created a new columns in the sessions dataframe. This reduces redundant information as the those properties of a session don’t change between transactions.

* It copied and marked “session_start” as a time index column into the new sessions dataframe to indicate the beginning of a session.

In [13]:
es["sessions"]

,session_id,device,customer_id,zip_code,session_start,join_date
1,1,desktop,2,13244,2014-01-01 00:00:00,2012-04-15 23:31:04
2,2,mobile,5,60091,2014-01-01 00:17:20,2010-07-17 05:27:50
3,3,mobile,4,60091,2014-01-01 00:28:10,2011-04-08 20:08:14
4,4,mobile,1,60091,2014-01-01 00:44:25,2011-04-17 10:48:33
5,5,mobile,4,60091,2014-01-01 01:11:30,2011-04-08 20:08:14
...,...,...,...,...,...,...
31,31,mobile,2,13244,2014-01-01 07:42:35,2012-04-15 23:31:04
32,32,mobile,5,60091,2014-01-01 08:02:05,2010-07-17 05:27:50
33,33,mobile,2,13244,2014-01-01 08:10:45,2012-04-15 23:31:04
34,34,desktop,3,13244,2014-01-01 08:24:50,2011-08-13 15:42:34


In [14]:
es["transactions"]

,transaction_id,session_id,transaction_time,product_id,amount,birthday
298,298,1,2014-01-01 00:00:00,5,127.64,1986-08-18
2,2,1,2014-01-01 00:01:05,2,109.48,1986-08-18
308,308,1,2014-01-01 00:02:10,3,95.06,1986-08-18
116,116,1,2014-01-01 00:03:15,4,78.92,1986-08-18
371,371,1,2014-01-01 00:04:20,3,31.54,1986-08-18
...,...,...,...,...,...,...
112,112,35,2014-01-01 08:56:15,5,55.42,2003-11-21
111,111,35,2014-01-01 08:57:20,3,34.87,2003-11-21
276,276,35,2014-01-01 08:58:25,1,10.94,2003-11-21
266,266,35,2014-01-01 08:59:30,5,19.86,2003-11-21


Note that transactions no longer includes the various sessions fields that were duplicated.

In [15]:
es = es.normalize_dataframe(
    base_dataframe_name="sessions",
    new_dataframe_name="customers",
    index="customer_id",
    make_time_index="join_date",
    additional_columns=["zip_code", "join_date"],
)

es

Entityset: customer_data
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 4]
    customers [Rows: 5, Columns: 3]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [16]:
es["customers"]

,customer_id,zip_code,join_date
5,5,60091,2010-07-17 05:27:50
4,4,60091,2011-04-08 20:08:14
1,1,60091,2011-04-17 10:48:33
3,3,13244,2011-08-13 15:42:34
2,2,13244,2012-04-15 23:31:04


In [17]:
es["sessions"]

,session_id,device,customer_id,session_start
1,1,desktop,2,2014-01-01 00:00:00
2,2,mobile,5,2014-01-01 00:17:20
3,3,mobile,4,2014-01-01 00:28:10
4,4,mobile,1,2014-01-01 00:44:25
5,5,mobile,4,2014-01-01 01:11:30
...,...,...,...,...
31,31,mobile,2,2014-01-01 07:42:35
32,32,mobile,5,2014-01-01 08:02:05
33,33,mobile,2,2014-01-01 08:10:45
34,34,desktop,3,2014-01-01 08:24:50


Zip code, join date are pulled out now and joined back via customer id.

In [ ]:
## Building features with an EntitySet

In [18]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name="products")

feature_matrix

,COUNT(transactions),MAX(transactions.amount),MEAN(transactions.amount),MIN(transactions.amount),SKEW(transactions.amount),STD(transactions.amount),SUM(transactions.amount),MODE(transactions.DAY(birthday)),MODE(transactions.DAY(transaction_time)),MODE(transactions.MONTH(birthday)),...,MODE(transactions.sessions.device),NUM_UNIQUE(transactions.DAY(birthday)),NUM_UNIQUE(transactions.DAY(transaction_time)),NUM_UNIQUE(transactions.MONTH(birthday)),NUM_UNIQUE(transactions.MONTH(transaction_time)),NUM_UNIQUE(transactions.WEEKDAY(birthday)),NUM_UNIQUE(transactions.WEEKDAY(transaction_time)),NUM_UNIQUE(transactions.YEAR(birthday)),NUM_UNIQUE(transactions.YEAR(transaction_time)),NUM_UNIQUE(transactions.sessions.device)
product_id,,,,,,,,,,,,,,,,,,,,,
1,102,149.56,73.429314,6.84,0.125525,42.479989,7489.79,18,1,7,...,desktop,4,1,3,1,4,1,5,1,3
2,92,149.95,76.319891,5.73,0.151934,46.336308,7021.43,18,1,8,...,desktop,4,1,3,1,4,1,5,1,3
3,96,148.31,73.001250,5.89,0.223938,38.871405,7008.12,18,1,8,...,desktop,4,1,3,1,4,1,5,1,3
4,106,146.46,76.311038,5.81,-0.132077,42.492501,8088.97,18,1,7,...,desktop,4,1,3,1,4,1,5,1,3
5,104,149.02,76.264904,5.91,0.098248,42.131902,7931.55,18,1,7,...,mobile,4,1,3,1,4,1,5,1,3


Note the constructed features are entirely dependent on the relationships within tables in the EntitySet.